# Lab 5:
## Topic 1 Binary Search Trees

A Binary Search Tree (BST) is a type of binary tree where each node has at most two children, and it follows a specific order:

  - Left subtree: Contains only nodes with values less than the parent node.
  - Right subtree: Contains only nodes with values greater than the parent node.

  This structure allows efficient searching, insertion, and deletion operations, usually in
𝑂(log𝑛) time for balanced trees.

Methods in a BST
Here are some common methods in a BST:

- **Insert**: Adds a new node to the tree while maintaining the BST property.
- **Search**: Finds if a given value exists in the tree.
- **Delete**: Removes a node from the tree and ensures the tree still satisfies the BST properties.
- **Traverse**:
  - In-order Traversal (Left, Root, Right): Visits nodes in ascending order.
  - Pre-order Traversal (Root, Left, Right): Useful for copying the tree.
  - Post-order Traversal (Left, Right, Root): Useful for deleting nodes or evaluating expressions.
- Find **Minimum** and **Maximum**: Finds the smallest and largest values in the BST.
- Find **Height**: Determines the height (or depth) of the tree, which is the longest path from the root to a leaf node.

In [ ]:
#@title Keeping some code here for displaying BST. Do not worry this code is not needed for your lab. You only need to run the code block.

def print_tree(root, val="val", left="left", right="right"):
    def display(root, val=val, left=left, right=right):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if getattr(root, right) is None and getattr(root, left) is None:
            line = '%s' % getattr(root, val)
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if getattr(root, right) is None:
            lines, n, p, x = display(getattr(root, left))
            s = '%s' % getattr(root, val)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if getattr(root, left) is None:
            lines, n, p, x = display(getattr(root, right))
            s = '%s' % getattr(root, val)
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = display(getattr(root, left))
        right, m, q, y = display(getattr(root, right))
        s = '%s' % getattr(root, val)
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

    lines, *_ = display(root, val, left, right)
    for line in lines:
        print(line)

Just like a LinkedList, a BST can start with a Node, which has a key, a value and a left and right child as pointers.

In [ ]:
class BinarySearchTreeNode:
    def __init__(self, value):
        """
        Initializes a node with the given value and sets left and right children to None.
        """
        self.value = value
        self.left = None  # Left child
        self.right = None  # Right child

The BST class:

In [ ]:
class BinarySearchTree:
    def __init__(self):
        """
        Initializes an empty Binary Search Tree with root set to None.
        """
        self.root = None

    def insert(self, value):
        """
        Inserts a new node with the specified value into the BST.
        """
        new_node = BinarySearchTreeNode(value)  # Create a new node with the given value
        if self.root is None:  # If tree is empty, set root to the new node
            self.root = new_node
        else:
            # Start from the root and find the correct spot for the new node
            current = self.root
            while True:
                if value < current.value:  # Go to the left subtree if value is smaller
                    if current.left is None:  # If left child is empty, insert here
                        current.left = new_node
                        break
                    else:
                        current = current.left  # Move to the left child
                else:  # Go to the right subtree if value is larger or equal
                    if current.right is None:  # If right child is empty, insert here
                        current.right = new_node
                        break
                    else:
                        current = current.right  # Move to the right child

    def search(self, value):
        """
        Searches for a node with the specified value in the BST.
        Returns True if found, otherwise False.
        """
        current = self.root  # Start from the root
        while current is not None:
            if value == current.value:  # Value found
                return True
            elif value < current.value:  # Go to the left subtree if value is smaller
                current = current.left
            else:  # Go to the right subtree if value is larger
                current = current.right
        return False  # Value not found

    def delete_node(self, node, val):
        """
        Recursively deletes a node with the specified value from the BST.
        Returns the modified tree (node).
        """
        if not node:
            return None  # Base case: if node is None, return None

        # Recursively find the node to delete
        if val < node.value:  # Go to the left subtree
            node.left = self.delete_node(node.left, val)
        elif val > node.value:  # Go to the right subtree
            node.right = self.delete_node(node.right, val)
        else:
            # Node to be deleted is found
            # Case 1: No children (leaf node)
            if not node.left and not node.right:
                node = None
            # Case 2: One child (either left or right)
            elif not node.left:  # Node has only a right child
                node = node.right
            elif not node.right:  # Node has only a left child
                node = node.left
            # Case 3: Two children
            else:
                # Find the in-order successor (smallest value in the right subtree)
                successor = node.right
                while successor.left:
                    successor = successor.left
                node.value = successor.value  # Replace current node's value with successor's value
                # Delete the successor node from the right subtree
                node.right = self.delete_node(node.right, successor.value)

        return node

    def delete(self, value):
        """
        Deletes a node with the specified value from the BST, starting from the root.
        """
        self.root = self.delete_node(self.root, value)

    def inorder_traversal(self):
        """
        Performs an in-order traversal of the BST iteratively.
        Prints the value of each node in ascending order (left-root-right).
        """
        stack = []
        current = self.root

        while stack or current:
            # Reach the leftmost node of the current node
            while current:
                stack.append(current)
                current = current.left

            # Current must be None at this point, pop the stack
            current = stack.pop()
            print(current.value)  # Print the node's value

            # Move to the right subtree
            current = current.right

    def print_tree(self):
        """
        Prints the binary search tree in a structured format.
        """
        if self.root is not None:
            print_tree(self.root, val="value", left="left", right="right")


####How does BST Insertion work?

Inserting a new node into a Binary Search Tree (BST) follows a specific process that maintains the BST property: left child nodes are less than the parent node, and right child nodes are greater than the parent node.
Steps for Inserting a Value in a BST
1. Start at the Root:

  If the tree is empty, set the new node as the root.
2. Traverse the Tree:

  Start at the root and compare the new value with the current node.
  Move left if the value is smaller, or right if it’s larger.
3. Find the Position:

  Continue left or right until reaching an empty spot (None).
4. Insert the Node:

  Place the new node in the empty spot as the left or right child of the last node visited.

Testing Insertion

In [ ]:
# Example usage:
bst = BinarySearchTree()
bst.insert(6)
bst.insert(2)
bst.insert(1)
bst.insert(4)
bst.insert(3)

print("Let's see how our tree is looking like after inserting these values (6,2,1,4,3):")
bst.print_tree()

print("Let's insert some more values 5,7,9 and 8")
bst.insert(5)
bst.insert(7)
bst.insert(9)
bst.insert(8)

bst.print_tree()

print("In order traversal of our BST")
bst.inorder_traversal()

Let's see how our tree is looking like after inserting these values (6,2,1,4,3):
  __6
 /   
 2_  
/  \ 
1  4 
  /  
  3  
Let's insert some more values 5,7,9 and 8
  ___6   
 /    \  
 2_   7_ 
/  \    \
1  4    9
  / \  / 
  3 5  8 
In order traversal of our BST
1
2
3
4
5
6
7
8
9


####How does BST Deletion work?
1. Locate the Node:

  Start at the root, move left or right depending on whether the value to delete is smaller or larger than the current node's value, until you find the node.
2. Handle Cases:

  - Case 1: Node has No Children (Leaf Node): Simply remove the node by setting its parent’s pointer to None.
  - Case 2: Node has One Child: Replace the node with its only child (left or right) by updating its parent’s pointer.
  - Case 3: Node has Two Children:

    Find the node’s in-order successor (smallest node in the right subtree).
    Replace the node’s value with the in-order successor’s value.
    Delete the in-order successor (which will either be a leaf or have one child).

Testing deletion or removal of node

In [ ]:
bst.delete(8)
print("After removing node 8:")
bst.print_tree()

print("Finding '8':", bst.search(8))  # Should print True for finding the value 6

# let's delete a node with 1 child: 7
bst.delete(7)
print("Removing node with 1 child: 7")
bst.print_tree()

# let's remove a node with 2 children: 4
bst.delete(4)
print("Removing node with 2 child: 4")
bst.print_tree()

# in order traversal
print("In order traversal of our BST")
bst.inorder_traversal()

After removing node 8:
  ___6  
 /    \ 
 2_   7 
/  \   \
1  4   9
  / \   
  3 5   
Finding '8': False
Removing node with 1 child: 7
  ___6 
 /    \
 2_   9
/  \   
1  4   
  / \  
  3 5  
Removing node with 2 child: 4
  __6 
 /   \
 2_  9
/  \  
1  5  
  /   
  3   
In order traversal of our BST
1
2
3
5
6
9


## Topic 2: Hash Tables

A hash table is a data structure that uses a hashing function to map keys to values for efficient data retrieval. It supports operations like insertion, deletion, and lookup, typically in constant time O(1) on average. Key terms:

- Hash Function: Converts a key into a specific index in an array, where the associated value is stored.
- Collision: When two keys hash to the same index, causing a conflict.

####Operations in Hash Tables
- Insertion: Add a key-value pair to the hash table.
- Search: Find the value associated with a key.
- Deletion: Remove a key-value pair from the hash table.

###Hash Table Implementation: Using Chaining

In [ ]:
class HashTableChaining:
    def __init__(self, size=10):
        """
        Initialize a hash table with the given size and an array of empty buckets (lists).
        """
        self.size = size
        self.table = [[] for _ in range(size)]

    def _hash(self, key):
        """
        Hash function: calculates index by taking modulo of the key's hash value.
        """
        return hash(key) % self.size

    def insert(self, key, value):
        """
        Insert a key-value pair into the hash table.
        """
        index = self._hash(key)
        # Update if the key exists
        for kv_pair in self.table[index]:
            if kv_pair[0] == key:
                kv_pair[1] = value
                return
        # Append if the key does not exist
        self.table[index].append([key, value])

    def search(self, key):
        """
        Search for a key in the hash table and return its value, or None if not found.
        """
        index = self._hash(key)
        for kv_pair in self.table[index]:
            if kv_pair[0] == key:
                return kv_pair[1]
        return None

    def delete(self, key):
        """
        Delete a key-value pair from the hash table.
        """
        index = self._hash(key)
        for i, kv_pair in enumerate(self.table[index]):
            if kv_pair[0] == key:
                del self.table[index][i]
                return True
        return False

    def __repr__(self):
        """
        Display the hash table for debugging and visualization.
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            table_str += f"Index {i}: {bucket}\n"
        return table_str


Testing the Hash Table

In [ ]:
print("Using a default hash table of size 10")
ht1 = HashTableChaining()
print(ht1.delete("Kiwifruit"))
ht1.insert("Apple", 58)
ht1.insert("Banana", 14)
ht1.insert("Cucumber", 8)
ht1.insert("Durian", 8)
ht1.insert("Elderberry", 8)
ht1.insert("Fig", 8)
print(ht1)

# Deleting items
print(ht1.delete("Banana"))
print(ht1)
print(ht1.delete("Carrot"))
print(ht1)
ht1.insert("Grapes", 8)
print(ht1)
ht1.insert("Honeydew", 8)
print(ht1)
ht1.insert("J", 8)
print(ht1)

Using a default hash table of size 10
False
Index 0: []
Index 1: [['Cucumber', 8]]
Index 2: [['Apple', 58], ['Durian', 8], ['Elderberry', 8]]
Index 3: []
Index 4: [['Banana', 14], ['Fig', 8]]
Index 5: []
Index 6: []
Index 7: []
Index 8: []
Index 9: []

True
Index 0: []
Index 1: [['Cucumber', 8]]
Index 2: [['Apple', 58], ['Durian', 8], ['Elderberry', 8]]
Index 3: []
Index 4: [['Fig', 8]]
Index 5: []
Index 6: []
Index 7: []
Index 8: []
Index 9: []

False
Index 0: []
Index 1: [['Cucumber', 8]]
Index 2: [['Apple', 58], ['Durian', 8], ['Elderberry', 8]]
Index 3: []
Index 4: [['Fig', 8]]
Index 5: []
Index 6: []
Index 7: []
Index 8: []
Index 9: []

Index 0: []
Index 1: [['Cucumber', 8]]
Index 2: [['Apple', 58], ['Durian', 8], ['Elderberry', 8]]
Index 3: []
Index 4: [['Fig', 8]]
Index 5: []
Index 6: []
Index 7: [['Grapes', 8]]
Index 8: []
Index 9: []

Index 0: []
Index 1: [['Cucumber', 8]]
Index 2: [['Apple', 58], ['Durian', 8], ['Elderberry', 8]]
Index 3: []
Index 4: [['Fig', 8]]
Index 5: []


### Hash Table using Linear Probing

In [ ]:
class HashTableProbingList:
    def __init__(self, size=10):
        """
        Initialize a hash table with the given size and an array of empty buckets (lists).
        Each bucket will hold a single key-value pair list, or remain empty.
        """
        self.size = size
        self.table = [[] for _ in range(size)]  # Each slot is an empty list to hold one key-value pair

    def _hash(self, key, probe=0):
        """
        Hash function with linear probing.
        Calculates the index by combining the base hash with the probe count.
        """
        base_hash = hash(key) % self.size
        return (base_hash + probe) % self.size  # Linear probing

    def insert(self, key, value):
        """
        Insert a key-value pair into the hash table using linear probing.
        If a collision occurs, it probes to find the next available slot.
        """
        for probe in range(self.size):
            index = self._hash(key, probe)
            # If the bucket is empty or has the key, insert or update
            if not self.table[index]:  # Empty list means an available slot
                self.table[index] = [key, value]
                return
            elif self.table[index][0] == key:  # Update the value if the key already exists
                self.table[index][1] = value
                return

        # If no empty slot is found after probing, raise an exception
        raise Exception("Hash table is full")

    def search(self, key):
        """
        Search for a key in the hash table and return its value, or None if not found.
        Uses linear probing to handle collisions.
        """
        for probe in range(self.size):
            index = self._hash(key, probe)
            if not self.table[index]:  # Empty slot means key is not in the table
                return None
            elif self.table[index][0] == key:  # Key found, return its value
                return self.table[index][1]
        return None  # Key not found after probing

    def delete(self, key):
        """
        Delete a key-value pair from the hash table using linear probing.
        Returns True if the key was found and deleted, or False if not found.
        """
        for probe in range(self.size):
            index = self._hash(key, probe)
            if not self.table[index]:  # Empty slot means key is not in the table
                return False
            elif self.table[index][0] == key:  # Key found, delete it by emptying the list
                self.table[index] = []
                return True
        return False  # Key not found after probing

    def __repr__(self):
        """
        Display the hash table for debugging and visualization.
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            table_str += f"Index {i}: {bucket}\n"
        return table_str


Testing the Hash Table with Probing on the same data:

In [ ]:
print("Using a default hash table of size 10")
ht1 = HashTableProbingList()
print(ht1.delete("Kiwifruit"))
ht1.insert("Apple", 58)
ht1.insert("Banana", 14)
ht1.insert("Cucumber", 8)
ht1.insert("Durian", 8)
ht1.insert("Elderberry", 8)
ht1.insert("Fig", 8)
print(ht1)

# Deleting items
print(ht1.delete("Banana"))
print(ht1)
print(ht1.delete("Carrot"))
print(ht1)
ht1.insert("Grapes", 8)
print(ht1)
ht1.insert("Honeydew", 8)
print(ht1)
ht1.insert("J", 8)
print(ht1)

Using a default hash table of size 10
False
Index 0: []
Index 1: ['Cucumber', 8]
Index 2: ['Apple', 58]
Index 3: ['Durian', 8]
Index 4: ['Banana', 14]
Index 5: ['Elderberry', 8]
Index 6: ['Fig', 8]
Index 7: []
Index 8: []
Index 9: []

True
Index 0: []
Index 1: ['Cucumber', 8]
Index 2: ['Apple', 58]
Index 3: ['Durian', 8]
Index 4: []
Index 5: ['Elderberry', 8]
Index 6: ['Fig', 8]
Index 7: []
Index 8: []
Index 9: []

False
Index 0: []
Index 1: ['Cucumber', 8]
Index 2: ['Apple', 58]
Index 3: ['Durian', 8]
Index 4: []
Index 5: ['Elderberry', 8]
Index 6: ['Fig', 8]
Index 7: []
Index 8: []
Index 9: []

Index 0: []
Index 1: ['Cucumber', 8]
Index 2: ['Apple', 58]
Index 3: ['Durian', 8]
Index 4: []
Index 5: ['Elderberry', 8]
Index 6: ['Fig', 8]
Index 7: ['Grapes', 8]
Index 8: []
Index 9: []

Index 0: []
Index 1: ['Cucumber', 8]
Index 2: ['Apple', 58]
Index 3: ['Durian', 8]
Index 4: []
Index 5: ['Elderberry', 8]
Index 6: ['Fig', 8]
Index 7: ['Grapes', 8]
Index 8: ['Honeydew', 8]
Index 9: []

Ind

To better understand how different collision resolution techniques work, compare the two hash table implementations (one using chaining and the other using linear probing) and visualize their differences.

## Practice Questions

1. **Find the $k$th Smallest Element in a BST**: Implement a function `find_kth_smallest(root, k)` that finds the $k$th smallest element in the BST. You must document your code and do algorithm analysis, mentioning the Time Complexity (Big O).

*Your explanation goes here.*

In [1]:
### Your solution code goes here. ###
def find_kth_smallest(root, k):
    """
    Finds the kth smallest element in a Binary Search Tree (BST).

    Args:
        root: The root node of the BST.
        k: The desired position (1-based) of the smallest element.

    Returns:
        The value of the kth smallest element, or None if not found.
    """

    count = 0
    result = None

    def inorder(node):
        nonlocal count, result

        if node is None or result is not None:
            return

        inorder(node.left)  # Traverse left subtree

        count += 1
        if count == k:  # Found the kth smallest
            result = node.value
            return

        inorder(node.right)  # Traverse right subtree

    inorder(root)
    return result

In [4]:
### Your test code goes here. ###

class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            self._insert_recursive(self.root, value)

    def _insert_recursive(self, node, value):
        if value < node.value:
            if node.left is None:
                node.left = Node(value)
            else:
                self._insert_recursive(node.left, value)
        else:  # value >= node.value
            if node.right is None:
                node.right = Node(value)
            else:
                self._insert_recursive(node.right, value)


# Example usage
bst = BinarySearchTree()
bst.insert(6)
bst.insert(2)
bst.insert(1)
bst.insert(4)
bst.insert(3)
bst.insert(5)
bst.insert(7)
bst.insert(9)
bst.insert(8)

k = 3
kth_smallest = find_kth_smallest(bst.root, k)
print(f"The {k}th smallest element is: {kth_smallest}")

The 3th smallest element is: 3


*Your analysis goes here.*

2. **Find the $k$th Largest Element in a BST**: Write a function `find_kth_largest(root, k)` to find the $k$th largest element in the BST. You must document your code and do algorithm analysis, mentioning the Time Complexity (Big O).

*Your explanation goes here.*

In [5]:
### Your solution code goes here. ###
def find_kth_largest(root, k):
    """
    Finds the kth largest element in a Binary Search Tree (BST).

    Args:
        root: The root node of the BST.
        k: The desired position (1-based) of the largest element.

    Returns:
        The value of the kth largest element, or None if not found.
    """

    count = 0
    result = None

    def reverse_inorder(node):
        nonlocal count, result

        if node is None or result is not None:
            return

        reverse_inorder(node.right)  # Traverse right subtree

        count += 1
        if count == k:  # Found the kth largest
            result = node.value
            return

        reverse_inorder(node.left)  # Traverse left subtree

    reverse_inorder(root)
    return result

In [6]:
### Your test code goes here. ###
# Example usage
bst = BinarySearchTree()
bst.insert(6)
bst.insert(2)
bst.insert(1)
bst.insert(4)
bst.insert(3)
bst.insert(5)
bst.insert(7)
bst.insert(9)
bst.insert(8)

k = 3
kth_largest = find_kth_largest(bst.root, k)
print(f"The {k}th largest element is: {kth_largest}")

The 3th largest element is: 7


*Your analysis goes here.*

3. **Convert a BST to a Reverse Sorted List**: Implement a function `bst_to_reverse_sorted_list(root)` that converts a BST into a sorted list in descending order without using extra space for the list (hint: post-order traversal).

*Your explanation goes here.*

In [7]:
### Your solution code goes here. ###
def bst_to_reverse_sorted_list(root):
    """
    Converts a Binary Search Tree (BST) into a reverse sorted list in-place.

    Args:
        root: The root of the BST.

    Returns:
        The head of the reverse sorted list (the node with the largest value).
    """

    head = None  # Initialize the head of the list

    def postorder(node):
        nonlocal head

        if node is None:
            return

        postorder(node.right)  # Traverse right subtree
        postorder(node.left)  # Traverse left subtree

        node.right = head  # Make current node's right point to the previous head
        node.left = None  # Set left pointer to None
        head = node  # Update head to the current node

    postorder(root)
    return head

In [8]:
### Your test code goes here. ###
# Example usage
bst = BinarySearchTree()
bst.insert(6)
bst.insert(2)
bst.insert(1)
bst.insert(4)
bst.insert(3)
bst.insert(5)
bst.insert(7)
bst.insert(9)
bst.insert(8)

head = bst_to_reverse_sorted_list(bst.root)

# Print the list (traversing from head to tail)
current = head
while current:
    print(current.value, end=" ")  # Output: 9 8 7 6 5 4 3 2 1
    current = current.right

6 2 1 4 3 5 7 9 8 

*Your analysis goes here.*

4. **Convert Sorted Array to BST**: Given a sorted array, write a function `sorted_array_to_bst(arr)` that constructs a balanced BST from the array.

*Your explanation goes here.*

In [15]:
### Your solution code goes here. ###
class BinarySearchTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def sorted_array_to_bst(arr):
    """
    Constructs a balanced Binary Search Tree (BST) from a sorted array.

    Args:
        arr: The sorted array.

    Returns:
        The root of the balanced BST.
    """

    if not arr:  # Base case: empty array
        return None

    mid = len(arr) // 2  # Find the middle index

    root = BinarySearchTreeNode(arr[mid])  # Create node with middle element

    root.left = sorted_array_to_bst(arr[:mid])  # Recursively build left subtree
    root.right = sorted_array_to_bst(arr[mid + 1:])  # Recursively build right subtree

    return root

In [17]:
### Your test code goes here. ###
class BinarySearchTree:
    def __init__(self, root=None):
        self.root = root

    def print_tree(self, node=None, level=0, prefix="Root: "):
        """Prints the tree structure with indentation."""
        if not node:
            node = self.root  # Start from root if node is not provided
        if node:
            print(" " * level + prefix + str(node.val))
            if node.left:
                self.print_tree(node.left, level + 1, "L--- ")
            if node.right:
                self.print_tree(node.right, level + 1, "R--- ")

# Example usage
arr = [-10, -3, 0, 5, 9]
root = sorted_array_to_bst(arr)

bst = BinarySearchTree()
bst.root = root  # Set the root of the BST

bst.print_tree()

Root: 0
 L--- -3
  L--- -10
 R--- 9
  L--- 5


*Your analysis goes here.*

5. **Hash Table with Your Hash Function**: You are to create a hash table with a size of 7 and make your own unique hash function. An example:

  $\text{hash}(\text{key}) = (\text{key} \% 7 + \text{probe}) \% 7$

  This hash function uses linear probing to handle collisions, where probe is the number of attempts made to find an empty slot in the table.

  Your task is to:

  - Insert the following keys in the hash table, in this order: 10, 20, 15, 7, 22 and show the Hash table status after insertion.
  - Explain in a few words how your hash function could affect the performance of operations (insertion, search, and deletion) in a hash table. Consider both time complexity and space utilization.


In [20]:
# YOUR CODE GOES HERE
def calculate_hash_value(key, probe):
  """Calculates the hash value for a given key and probe.
  """
  hash_value = (key % 7 + 3 * probe) % 7
  return hash_value

# Example Usage
key = 10  # Assign a value to 'key'
probe = 2   # Assign a value to 'probe'

hash_value = calculate_hash_value(key, probe) # calling the hash function with key and probe and assigning it to the variable hash_value
print(hash_value) # Print the hash value

2
